In [1]:
import requests
import torch
import torch.nn as nn
import os
from torchvision import models, datasets, transforms
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchattacks

from sklearn.model_selection import train_test_split
from typing import Tuple

import warnings
from tqdm import tqdm



warnings.filterwarnings('ignore')

### TOKEN  ###
TOKEN = "REDACTED" # to be changed according to your token (given to you for the assignments via email)

## ALLOWED MODELS
allowed_models = {
    "resnet18": models.resnet18,
    "resnet34": models.resnet34,
    "resnet50": models.resnet50,
}


In [ ]:
os.makedirs("out/models", exist_ok=True)

In [3]:
# Dataset Class with RGB Conversion
class TaskDataset(Dataset):
    def __init__(self, ids, imgs, labels, transform=None):
        self.ids = ids
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
        
    def __getitem__(self, index) -> Tuple[int, torch.Tensor, int]:
        id_ = self.ids[index]
        img = self.imgs[index]

        # Task conversion
        if img.mode != 'RGB':
            img = img.convert('RGB')
        
        if not self.transform is None:
            img = self.transform(img)
            
        label = self.labels[index]
        
        return id_, img, label

    def __len__(self):
        return len(self.ids)

In [4]:
# CPU Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Data Transformation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((32, 32))
    # transforms.Normalize(mean, std)
])

# Load train.pt
try:
    train_data : TaskDataset= torch.load("Train.pt", map_location=device, weights_only=False)
    print(f"Training dataset: {len(train_data)} samples")
    
except Exception as error:
    print(f"Error loading model: {error}")
    exit(1)

Using device: cpu
Training dataset: 100000 samples


In [65]:
# Training hyperparameters
EPOCHS = 30
LEARNING_RATE = 0.09
WEIGHT_DECAY = 1e-4
FGSM_EPSILON = 0.03 # FGSM Attack strength
PGD_EPSILON = 0.03 # PGD Attack Strength
ALPHA = 0.007   # PGD step size
PGD_ITER = 5   # Number of Interattions in PGD
RESNET_MODEL = "resnet18"  # Resnet Model settings

In [66]:
def adversarial_train_epoch(model, train_loader, optimizer, device, fgsm, pgd):
    """
    Train one epoch with adversarial training
    
    Args:
        model: Neural network model
        train_loader: Training data loader
        optimizer: Optimizer
        device: Device to run on
        fgsm: FGSM Torchattack Object
        pgd: PGD Torchattack Object
    Returns:
        Average loss and accuracy for the epoch
    """
    model.train()
    criterion = nn.CrossEntropyLoss()
    total_loss = 0
    correct = 0
    total = 0

    batch_tqdm = tqdm(train_loader, desc='Training', leave=False)

    for _, batch_images, batch_labels in batch_tqdm:
        batch_images, batch_labels = batch_images.to(device), torch.as_tensor(batch_labels).to(device)

        batch_images = torch.clamp(batch_images, 0, 1)

         # Generate adversarial examples
        fgsm_images = fgsm(batch_images, batch_labels)
        pgd_images = pgd(batch_images, batch_labels)
        
        # Combine clean, FGSM, PGD examples
        combined_images = torch.cat([batch_images, fgsm_images, pgd_images])
        combined_labels = torch.cat([batch_labels, batch_labels, batch_labels])

        # Forward pass
        outputs = model(combined_images)
        loss = criterion(outputs, combined_labels)
        optimizer.zero_grad()

        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Statistics
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += combined_labels.size(0)
        correct += predicted.eq(combined_labels).sum().item()

        # Update progress bar
        batch_tqdm.set_postfix({
            'Loss': f'{loss.item():.4f}',
            'Acc': f'{100.*correct/total:.2f}%'
        })

    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total
    
    return avg_loss, accuracy


In [67]:
def evaluate_robustness(model, val_loader, device, attack=None):
    """
    Evaluate model robustness on clean or adversarial examples
    
    Args:
        model: Neural network model
        val_loader: Validation data loader
        device: Device to run on
        attack: None for clean, else provide the attack function - fgsm or pgd
        
    Returns:
        Accuracy percentage
    """
    model.eval()
    correct = 0
    total = 0

    batch_tqdm = tqdm(val_loader, desc=f'{attack.__class__.__name__ if attack else "Clean"}', leave=False)

    for _, batch_images, batch_labels in batch_tqdm:
        batch_images, batch_labels = batch_images.to(device), batch_labels.to(device)

        batch_images = torch.clamp(batch_images, 0, 1)
        
        # Generate adversarial examples if requested
        if attack:
            adversarial_images = attack(batch_images, batch_labels)
            
        with torch.no_grad():
            # Forward pass - based on attack type - adversarial or clean
            outputs = model(adversarial_images) if attack else model(batch_images)
            _, predicted = outputs.max(1)
            total += batch_labels.size(0)
            correct += predicted.eq(batch_labels).sum().item()

            # Update progress bar
            batch_tqdm.set_postfix({
                f'{attack.__class__.__name__ if attack else "Clean"} Acc': f'{100.*correct/total:.2f}%'
            })

    
    accuracy = 100. * correct / total
    return accuracy

In [68]:
ids = [train_data[i][0] for i in range(len(train_data))]
imgs = [train_data[i][1] for i in range(len(train_data))]
labels = [train_data[i][2] for i in range(len(train_data))]

# Split using sklearn
ids_train, ids_val, imgs_train, imgs_val, labels_train, labels_val = train_test_split(
    ids, imgs, labels, test_size=0.2, random_state=42, stratify=labels
)

# unified train and val datasets
train_dataset = TaskDataset(ids_train, imgs_train, labels_train, transform=transform)
val_dataset = TaskDataset(ids_val, imgs_val, labels_val, transform=transform)
    
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)


In [69]:
def train_and_evaluate():
    """
    Trains and evaluates a model on 20 epochs

    Returns:
        tuple: A tuple containing best model state dictionary and clean accuracy
    """
    # Initialize model
    resnet_model = allowed_models[RESNET_MODEL](weights=None)
    resnet_model.fc = nn.Linear(resnet_model.fc.weight.shape[1], 10)
    resnet_model = resnet_model.to(device)

    # Initialize optimizer and scheduler
    optimizer = optim.SGD(resnet_model.parameters(), lr=LEARNING_RATE,
                         momentum=0.9, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.5)

    # Initialize attacks
    fgsm = torchattacks.FGSM(resnet_model, eps=FGSM_EPSILON)
    pgd = torchattacks.PGD(resnet_model, eps=PGD_EPSILON, alpha=ALPHA, steps=PGD_ITER)

    # Store best clean accuracy and model state
    best_clean_acc = 0
    best_model_state = None

    print(f"Training with parameters: LR={LEARNING_RATE}, FGSM Eps={FGSM_EPSILON}, PGD Eps={PGD_EPSILON}, Alpha={ALPHA}, PGD_Iter={PGD_ITER}")
    # Training loop
    for epoch in range(EPOCHS):
        print(f"\nEpoch {epoch+1}/{EPOCHS}")
        # Training phase
        train_loss, train_acc = adversarial_train_epoch(resnet_model, train_loader, optimizer, device, fgsm, pgd)

        # Evaluation phase
        clean_acc = evaluate_robustness(resnet_model, val_loader, device, attack=None)
        fgsm_acc = evaluate_robustness(resnet_model, val_loader, device, attack=fgsm)
        pgd_acc = evaluate_robustness(resnet_model, val_loader, device, attack=pgd)

        # Update learning rate
        scheduler.step()

        # Save best model based on clean accuracy
        if clean_acc > best_clean_acc and clean_acc > 50:
            best_clean_acc = clean_acc
            best_model_state = resnet_model.state_dict().copy()
            torch.save(best_model_state, os.path.join("out/models", f"{RESNET_MODEL}_{LEARNING_RATE}_{best_clean_acc}_{epoch+1}.pt"))
        
            print(f"New best clean accuracy: {clean_acc:.2f}%")
    
        # Print epoch summary
        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"Clean Acc: {clean_acc:.2f}% | FGSM Acc: {fgsm_acc:.2f}% | PGD Acc: {pgd_acc:.2f}%")
        print(f"Current LR: {scheduler.get_last_lr()[0]:.6f}")


    return best_model_state, best_clean_acc

In [70]:
# Train and evaluate the model
best_model_state, best_clean_acc = train_and_evaluate()

Training with parameters: LR=0.09, FGSM Eps=0.03, PGD Eps=0.03, Alpha=0.007, PGD_Iter=5

Epoch 1/30


Train Loss: 1.8037 | Train Acc: 33.82%
Clean Acc: 44.01% | FGSM Acc: 32.26% | PGD Acc: 32.69%
Current LR: 0.090000

Epoch 2/30


Train Loss: 1.6797 | Train Acc: 36.74%
Clean Acc: 46.99% | FGSM Acc: 35.20% | PGD Acc: 36.35%
Current LR: 0.090000

Epoch 3/30


Train Loss: 1.6540 | Train Acc: 38.55%
Clean Acc: 47.91% | FGSM Acc: 32.80% | PGD Acc: 34.41%
Current LR: 0.090000

Epoch 4/30


Train Loss: 1.6371 | Train Acc: 39.34%
Clean Acc: 44.80% | FGSM Acc: 33.34% | PGD Acc: 34.48%
Current LR: 0.090000

Epoch 5/30


...

Train Loss: 0.8668 | Train Acc: 68.52%
Clean Acc: 60.74% | FGSM Acc: 83.10% | PGD Acc: 33.17%
Current LR: 0.005625

Epoch 27/30


Train Loss: 0.8619 | Train Acc: 68.70%
Clean Acc: 56.72% | FGSM Acc: 77.34% | PGD Acc: 33.95%
Current LR: 0.005625

Epoch 28/30


Train Loss: 0.8496 | Train Acc: 69.17%
Clean Acc: 58.78% | FGSM Acc: 82.02% | PGD Acc: 31.20%
Current LR: 0.005625

Epoch 29/30


Train Loss: 0.8463 | Train Acc: 69.42%
Clean Acc: 59.41% | FGSM Acc: 83.31% | PGD Acc: 33.84%
Current LR: 0.005625

Epoch 30/30


Train Loss: 0.8303 | Train Acc: 69.99%
Clean Acc: 58.12% | FGSM Acc: 80.28% | PGD Acc: 32.73%
Current LR: 0.002812


In [71]:
# Save for submission
os.makedirs("out/models", exist_ok=True)
filename = f"model_{RESNET_MODEL}_{LEARNING_RATE}_{best_clean_acc}.pt"
filepath = os.path.join("out/models", filename)

# Save model
torch.save(best_model_state, filepath)
print(f"Saved model_{RESNET_MODEL}_{LEARNING_RATE}_{best_clean_acc}.pt to {filepath}")

Saved model_resnet18_0.09_61.34.pt to out/models/model_resnet18_0.09_61.34.pt


In [72]:
# Validate Model before SUbmission

with open(filepath, "rb") as f:
    try:
        model: torch.nn.Module = allowed_models[RESNET_MODEL](weights=None)
        model.fc = torch.nn.Linear(model.fc.weight.shape[1], 10)
    except Exception as e:
        raise Exception(
            f"Invalid model class, {e=}, only {allowed_models.keys()} are allowed",
        )
    try:
        state_dict = torch.load(f, map_location=torch.device("cpu"))
        model.load_state_dict(state_dict, strict=True)
        model.eval()
        out = model(torch.randn(1, 3, 32, 32))
    except Exception as e:
        raise Exception(f"Invalid model, {e=}")

    assert out.shape == (1, 10), "Invalid output shape"


In [58]:

# Send the model to the server, replace the string "TOKEN" with the string of token provided to you
response = requests.post(
    "http://34.122.51.94:9090/robustness",
    files={"file": open(filepath, "rb")},
    headers={"token": TOKEN, "model-name": RESNET_MODEL}
)

# Should be 400, the clean accuracy is too low
print(response.json())


{'clean_accuracy': 0.584, 'fgsm_accuracy': 0.701, 'pgd_accuracy': 0.31833333333333336}
